# Model Definition and Evaluation Overview

This notebook provides an overview of the **Model Definition** and **Evaluation** process for a stock price prediction model using LSTM (Long Short-Term Memory) networks. The model is built, trained, and evaluated with performance metrics.

## 1. Model Definition

### Building the LSTM Model
The function `build_lstm_model` constructs an LSTM model based on the specified hyperparameters:

```python
def build_lstm_model(window_size, num_features, best_params):
    units = best_params['units']
    dropout_rate = best_params['dropout_rate']
    learning_rate = best_params['learning_rate']
    batch_size = best_params['batch_size']

    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(window_size, num_features)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model
```

### Training the Model
The function `train_lstm_model` trains the LSTM model with training data and uses callbacks for early stopping and learning rate reduction:

```python
def train_lstm_model(model, x_train, y_train, epochs, batch_size):
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001)
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping, reduce_lr])
    return history
```

### Hyperparameter Tuning
The `objective` function defines the search space for hyperparameters and evaluates the model’s performance using Mean Absolute Error (MAE):

```python
def objective(trial, window_size, num_features, x_train, y_train, close_values, scaler, close_scaler, close_data):
    units = trial.suggest_int('units', 50, 200)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(window_size, num_features)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    history = model.fit(x_train, y_train, epochs=100, batch_size=batch_size, verbose=0, callbacks=[early_stopping, reduce_lr])

    x_test = prepare_test_data(close_values, scaler, window_size)
    predictions = make_predictions(model, x_test, close_scaler)

    mae = calculate_mae(predictions, close_data['Close'].values[window_size:])
    return mae
```

## 2. Model Evaluation

### Preparing Test Data
The function `prepare_test_data` scales the test data and prepares it for making predictions:

```python
def prepare_test_data(data, scaler, window_size):
    scaled_data = scaler.transform(data)
    x_test = [scaled_data[i-window_size:i, :] for i in range(window_size, len(scaled_data))]
    x_test = np.array(x_test)
    return x_test
```

### Making Predictions
The function `make_predictions` generates predictions using the trained model and inverse-transforms them to the original scale:

```python
def make_predictions(model, x_test, close_scaler):
    predictions = [close_scaler.inverse_transform([[model.predict(np.reshape(x_input, (1, x_input.shape[0], x_input.shape[1])))[0, 0]]])[0, 0] for x_input in x_test]
    return np.array(predictions)
```

### Evaluating the Model
The following functions assess the model’s performance using evaluation metrics:

- **Directional Accuracy (DA)**
  ```python
  def calculate_da(predictions, true_values):
      min_len = min(len(predictions), len(true_values))
      correct_directions = sum(1 for i in range(1, min_len) if np.sign(predictions[i] - predictions[i-1]) == np.sign(true_values[i] - true_values[i-1]))
      directional_accuracy = correct_directions / (min_len - 1)
      return directional_accuracy
  ```

- **Mean Absolute Error (MAE)**
  ```python
  def calculate_mae(predictions, true_values):
      min_len = min(len(predictions), len(true_values))
      mae = np.mean(np.abs(predictions[:min_len] - true_values[:min_len]))
      return mae
  ```

### Integrating and Running the Main Process
The `main` function integrates all components, runs the training and evaluation, and prints performance metrics:

```python
def main():
    try:
        ticker = input("Enter the stock symbol (e.g., AAPL): ").upper()
        start_date = '2021-01-01'
        end_date = '2024-01-01'
        window_size = 60
        epochs = 100

        data = download_stock_data(ticker, start_date, end_date)
        plot_close_price_history(data)
        plot_candlestick_chart(data)

        close_data = data.filter(['Close', 'Volume'])
        close_data = add_technical_indicators(close_data)
        close_data = close_data.dropna()

        close_values = close_data.values
        x_train, y_train, scaler, close_scaler = prepare_training_data(close_values, window_size)
        num_features = close_values.shape[1]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, window_size, num_features, x_train, y_train, close_values, scaler, close_scaler, close_data), n_trials=10)

        best_params = study.best_params
        model = build_lstm_model(window_size, num_features, best_params)
        history = train_lstm_model(model, x_train, y_train, epochs, best_params['batch_size'])

        x_test = prepare_test_data(close_values, scaler, window_size)
        predictions = make_predictions(model, x_test, close_scaler)

        da = calculate_da(predictions, close_data['Close'].values[window_size:])
        print(f"Directional Accuracy (DA): {da}")

        mae = calculate_mae(predictions, close_data['Close'].values[window_size:])
        print(f"Mean Absolute Error (MAE): {mae}")

        print("Final predicted price:", predictions[-1])

    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    main()
```

This notebook provides a comprehensive view of the model definition and evaluation process. Use it to understand the model's construction, training, and performance assessment.